In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [ ]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [ ]:
# !./setuprds.sh
!./setupdynamo.sh

In [26]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring service s3 to use http://localh

In [3]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [ ]:
# Setup the RDS roles
role_payload = { "type": "CREATE_ROLES", "credentials": {
        "account_api_worker": "pwd_for_account_api",
        "save_tx_api_worker": "pwd_for_transaction_api",
        "float_api_worker": "pwd_for_float_api",
        "auth_api_worker": "pwd_for_auth_api_worker"
    }
}
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps(role_payload))

In [ ]:
# Setup the RDS tables
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps({'type': 'SETUP_TABLES'}))

In [4]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [5]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [6]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [7]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='account-create', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [8]:
def generate_saving_transaction(account_id, ref_amount = 100, client_id = 'zar_client_co', float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id,
        'clientId': client_id
    }
    return test_saving_dict

In [17]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='activity-save', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']['amount']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [23]:
def run_accrual_for_float(client_id = 'zar_client_co', float_id = 'zar_cash_primary', accrued_amount = 100):
    accrualDict = {
        'clientId': client_id,
        'floatId': float_id,
        'accrualAmount': accrued_amount,
        'currency': 'ZAR',
        'unit': 'HUNDREDTH_CENT',
        'backingEntityIdentifier': 'tx-id-backing'
    }
    lambda_result = local_lambda.invoke(FunctionName='float-accrue', InvocationType='RequestResponse', 
                                               Payload=json.dumps(accrualDict, default=str))
    lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
    return lambda_payload

In [25]:
print_deployed_functions()

Function list:  []


In [12]:
accounts = create_number_accounts(number_accounts = 10)

Account with ID fff737dc-f8f3-48a9-9e9a-c425b54da2ea persisted at 2019-07-02T08:38:14.834Z
Account with ID 1383185b-ae19-4b50-b4c9-e78a9fd89392 persisted at 2019-07-02T08:38:16.940Z
Account with ID 504d76f6-5c28-4f38-907e-877c53efc124 persisted at 2019-07-02T08:38:17.650Z
Account with ID e72e04ec-d5e7-48e9-868a-22d0a27eff2a persisted at 2019-07-02T08:38:18.315Z
Account with ID 43322c8c-1afe-4bd8-9c89-d114a2053dba persisted at 2019-07-02T08:38:18.993Z
Account with ID a317406a-89c5-49a3-bc39-bf5d639641d4 persisted at 2019-07-02T08:38:19.681Z
Account with ID ea4af51b-cf21-4587-9c7d-9d3ea6132a8f persisted at 2019-07-02T08:38:20.349Z
Account with ID f31e753e-dfa9-4057-b824-5084163d0b8d persisted at 2019-07-02T08:38:21.012Z
Account with ID eb1326db-23e7-441a-867a-d2a4948d623b persisted at 2019-07-02T08:38:21.643Z
Account with ID db2d3721-8d86-4938-a10e-784255fbbaf9 persisted at 2019-07-02T08:38:22.263Z


In [13]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

Name                AccountId
------------------  ------------------------------------
Ben Carrillo        fff737dc-f8f3-48a9-9e9a-c425b54da2ea
Robert Dixon        1383185b-ae19-4b50-b4c9-e78a9fd89392
Daniel Snell        504d76f6-5c28-4f38-907e-877c53efc124
Charles Johnson     e72e04ec-d5e7-48e9-868a-22d0a27eff2a
Ann Graham          43322c8c-1afe-4bd8-9c89-d114a2053dba
Lee Roes            a317406a-89c5-49a3-bc39-bf5d639641d4
Joe Alonzo          ea4af51b-cf21-4587-9c7d-9d3ea6132a8f
Angelia Montemurro  f31e753e-dfa9-4057-b824-5084163d0b8d
Rebecca Bridges     eb1326db-23e7-441a-867a-d2a4948d623b
Kim Boyce           db2d3721-8d86-4938-a10e-784255fbbaf9


In [15]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='activity-save', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

Lambda result {'body': '{"transactionDetails":[[{"transaction_id":"3f70cc91-df84-496e-b842-11082a294f74","creation_time":"2019-07-02T08:43:30.659Z"}],[{"transaction_id":"cea1dde7-df71-43a2-8bff-f125345ac6e2","creation_time":"2019-07-02T08:43:30.659Z"},{"transaction_id":"cea1dde7-df71-43a2-8bff-f125345ac6e2","creation_time":"2019-07-02T08:43:30.659Z"}]],"newBalance":{"amount":7017,"unit":"HUNDREDTH_CENT"}}', 'statusCode': 200}


In [27]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

Tx for account fff737dc-f8f3-48a9-9e9a-c425b54da2ea persisted with new balance 353831911
Tx for account 1383185b-ae19-4b50-b4c9-e78a9fd89392 persisted with new balance 274950108
Tx for account 504d76f6-5c28-4f38-907e-877c53efc124 persisted with new balance 226013752
Tx for account e72e04ec-d5e7-48e9-868a-22d0a27eff2a persisted with new balance 377469579
Tx for account 43322c8c-1afe-4bd8-9c89-d114a2053dba persisted with new balance 215532375
Tx for account a317406a-89c5-49a3-bc39-bf5d639641d4 persisted with new balance 393586622
Tx for account ea4af51b-cf21-4587-9c7d-9d3ea6132a8f persisted with new balance 345240381
Tx for account f31e753e-dfa9-4057-b824-5084163d0b8d persisted with new balance 245207361
Tx for account eb1326db-23e7-441a-867a-d2a4948d623b persisted with new balance 293749389
Tx for account db2d3721-8d86-4938-a10e-784255fbbaf9 persisted with new balance 392296515
Tx for account fff737dc-f8f3-48a9-9e9a-c425b54da2ea persisted with new balance 439418404
Tx for account 138318

In [28]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [29]:
merged_df = merge_dataframes(accounts, account_balances)

In [30]:
merged_df

,accountId,floatId,Holder Name,balanceRand
0,fff737dc-f8f3-48a9-9e9a-c425b54da2ea,zar_cash_float,Ben Carrillo,44789.0095
1,1383185b-ae19-4b50-b4c9-e78a9fd89392,zar_cash_float,Robert Dixon,37991.1250
2,504d76f6-5c28-4f38-907e-877c53efc124,zar_cash_float,Daniel Snell,29637.3683
3,e72e04ec-d5e7-48e9-868a-22d0a27eff2a,zar_cash_float,Charles Johnson,49879.4696
4,43322c8c-1afe-4bd8-9c89-d114a2053dba,zar_cash_float,Ann Graham,31387.5444
5,a317406a-89c5-49a3-bc39-bf5d639641d4,zar_cash_float,Lee Roes,49602.8392
6,ea4af51b-cf21-4587-9c7d-9d3ea6132a8f,zar_cash_float,Joe Alonzo,40550.9581
7,f31e753e-dfa9-4057-b824-5084163d0b8d,zar_cash_float,Angelia Montemurro,34084.3813
8,eb1326db-23e7-441a-867a-d2a4948d623b,zar_cash_float,Rebecca Bridges,41432.7656
9,db2d3721-8d86-4938-a10e-784255fbbaf9,zar_cash_float,Kim Boyce,51652.0691


In [31]:
amount_to_accrue = round(random.random() * 100 * 100 * 100)
print('Instructing accrual of : R', amount_to_accrue / (100 * 100))
accrual_result = run_accrual_for_float(accrued_amount = amount_to_accrue)
allocation_details = json.loads(accrual_result['userAllocationTransactions']['body'])
account_transaction_list = allocation_details['allocationRecords']['accountTxIds']
# print(tabulate.tabulate(account_transaction_list, headers = 'keys'))

Instructing accrual of : R 28.5305


In [32]:
# print(accrual_result)
account_transactions = allocation_details['allocationRecords']['accountTxIds']
number_of_float_transactions = len(allocation_details['allocationRecords']['floatTxIds'])
number_of_account_transactions = len(account_transactions)
print('%d float transactions, %d account transactions' % (number_of_float_transactions, number_of_account_transactions))
sum_of_accounts = sum([transaction['amount'] for transaction in account_transactions])
print('Sum of account additions: ', sum_of_accounts)
bonus_allocation = accrual_result['entityAllocations']['bonusShare']
company_allocation = accrual_result['entityAllocations']['clientShare']
print('Bonus allocation: %d, and client co allocation: %d' % (bonus_allocation, company_allocation))
total_allocations = sum_of_accounts + bonus_allocation + company_allocation
print('Of accrual %d, allocations %d' % (amount_to_accrue, total_allocations))
rounding_allocated_to_bonus = isinstance(allocation_details['bonusAllocation'], list)
print('Was there a rounding allocation? : ', rounding_allocated_to_bonus)
print('Rounding allocation record: ', allocation_details['bonusAllocation'])

11 float transactions, 11 account transactions
Sum of account additions:  228243
Bonus allocation: 28531, and client co allocation: 28531
Of accrual 285305, allocations 285305
Was there a rounding allocation? :  False
Rounding allocation record:  {}


In [34]:
balance_payload = { 
    'accountId': accounts[0]['accountId'], 
    'clientId': 'zar_client_co', 
    'floatId': 'zar_cash_primary',
    'currency': 'ZAR',
    'timezone': 'Africa/Johannesburg'
}
balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
projection_decoded = decode_lambda_result(balance_projection)
print(projection_decoded)

{'balanceEndOfToday': {'amount': 496419132, 'currency': 'ZAR', 'datetime': '2019-07-02T23:59:59+02:00', 'epochMilli': 1562104799999, 'timezone': 'Africa/Johannesburg', 'unit': 'HUNDREDTH_CENT'}, 'balanceSubsequentDays': [{'amount': 496485775, 'currency': 'ZAR', 'datetime': '2019-07-03T23:59:59+02:00', 'epochMilli': 1562191199999, 'timezone': 'Africa/Johannesburg', 'unit': 'HUNDREDTH_CENT'}, {'amount': 496552427, 'currency': 'ZAR', 'datetime': '2019-07-04T23:59:59+02:00', 'epochMilli': 1562277599999, 'timezone': 'Africa/Johannesburg', 'unit': 'HUNDREDTH_CENT'}, {'amount': 496619087, 'currency': 'ZAR', 'datetime': '2019-07-05T23:59:59+02:00', 'epochMilli': 1562363999999, 'timezone': 'Africa/Johannesburg', 'unit': 'HUNDREDTH_CENT'}, {'amount': 496685756, 'currency': 'ZAR', 'datetime': '2019-07-06T23:59:59+02:00', 'epochMilli': 1562450399999, 'timezone': 'Africa/Johannesburg', 'unit': 'HUNDREDTH_CENT'}, {'amount': 496752434, 'currency': 'ZAR', 'datetime': '2019-07-07T23:59:59+02:00', 'epoc

In [39]:
def fetch_account_balance(accountId, clientId = 'zar_client_co', floatId = 'zar_cash_primary', currency = 'ZAR', timezone = 'Africa/Johannesburg'):
    balance_payload = { 
        'accountId': accounts[0]['accountId'], 
        'clientId': 'zar_client_co', 
        'floatId': 'zar_cash_primary',
        'currency': 'ZAR',
        'timezone': 'Africa/Johannesburg'
    }
    balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
    projection_decoded = decode_lambda_result(balance_projection)
    return projection_decoded

In [56]:
def fetch_account_balances(accounts):
#     balance_dicts = [fetch_account_balance(account['accountId']) for account in accounts]
    balance_dicts = []
    for account in accounts:
#         print('account: ', account)
        balance_dict = fetch_account_balance(account['accountId'])
        balance_dict['accountId'] = account['accountId']
        projected_dict = {
            'accountId': balance_dict['accountId'],
            'currentBalanceZAR': balance_dict['currentBalance']['amount'] / 10000,
            'balanceEndOfDayZAR': balance_dict['balanceEndOfToday']['amount'] / 10000,
            'balancedNextDayZAR': balance_dict['balanceSubsequentDays'][0]['amount'] / 10000
        }
        balance_dicts.append(projected_dict)
    
    return balance_dicts;

In [59]:
balances_projections = fetch_account_balances(accounts)

In [60]:
balance_df = pd.DataFrame(balances_projections)
balance_df

,accountId,balanceEndOfDayZAR,balancedNextDayZAR,currentBalanceZAR
0,fff737dc-f8f3-48a9-9e9a-c425b54da2ea,49641.9132,49648.5775,49635.2498
1,1383185b-ae19-4b50-b4c9-e78a9fd89392,49641.9132,49648.5775,49635.2498
2,504d76f6-5c28-4f38-907e-877c53efc124,49641.9132,49648.5775,49635.2498
3,e72e04ec-d5e7-48e9-868a-22d0a27eff2a,49641.9132,49648.5775,49635.2498
4,43322c8c-1afe-4bd8-9c89-d114a2053dba,49641.9132,49648.5775,49635.2498
5,a317406a-89c5-49a3-bc39-bf5d639641d4,49641.9132,49648.5775,49635.2498
6,ea4af51b-cf21-4587-9c7d-9d3ea6132a8f,49641.9132,49648.5775,49635.2498
7,f31e753e-dfa9-4057-b824-5084163d0b8d,49641.9132,49648.5775,49635.2498
8,eb1326db-23e7-441a-867a-d2a4948d623b,49641.9132,49648.5775,49635.2498
9,db2d3721-8d86-4938-a10e-784255fbbaf9,49641.9132,49648.5775,49635.2498


In [61]:
print(balances_projections)

[{'accountId': 'fff737dc-f8f3-48a9-9e9a-c425b54da2ea', 'currentBalanceZAR': 49635.2498, 'balanceEndOfDayZAR': 49641.9132, 'balancedNextDayZAR': 49648.5775}, {'accountId': '1383185b-ae19-4b50-b4c9-e78a9fd89392', 'currentBalanceZAR': 49635.2498, 'balanceEndOfDayZAR': 49641.9132, 'balancedNextDayZAR': 49648.5775}, {'accountId': '504d76f6-5c28-4f38-907e-877c53efc124', 'currentBalanceZAR': 49635.2498, 'balanceEndOfDayZAR': 49641.9132, 'balancedNextDayZAR': 49648.5775}, {'accountId': 'e72e04ec-d5e7-48e9-868a-22d0a27eff2a', 'currentBalanceZAR': 49635.2498, 'balanceEndOfDayZAR': 49641.9132, 'balancedNextDayZAR': 49648.5775}, {'accountId': '43322c8c-1afe-4bd8-9c89-d114a2053dba', 'currentBalanceZAR': 49635.2498, 'balanceEndOfDayZAR': 49641.9132, 'balancedNextDayZAR': 49648.5775}, {'accountId': 'a317406a-89c5-49a3-bc39-bf5d639641d4', 'currentBalanceZAR': 49635.2498, 'balanceEndOfDayZAR': 49641.9132, 'balancedNextDayZAR': 49648.5775}, {'accountId': 'ea4af51b-cf21-4587-9c7d-9d3ea6132a8f', 'currentB